In [1]:
import tensorflow as tf
import os

In [3]:
train_dir = "/Users/ttdat/Documents/Do_An_Co_So/code/code_final/data_split/train"
test_dir = "/Users/ttdat/Documents/Do_An_Co_So/code/code_final/data_split/test"
img_size = (160, 160)
batch_size = 32

In [4]:
# Load ảnh thành dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'  # vì softmax
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

Found 448 files belonging to 26 classes.
Found 207 files belonging to 26 classes.


2025-05-10 18:29:42.717982: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-05-10 18:29:42.718169: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-10 18:29:42.718189: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-10 18:29:42.718404: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-10 18:29:42.718421: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
# Tối ưu hiệu suất
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [6]:
# Mô hình MobileNetV2 + phân lớp
base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(train_ds.element_spec[1].shape[-1], activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# Huấn luyện
model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10


2025-05-10 18:30:47.106059: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 229ms/step - accuracy: 0.2135 - loss: 3.6894 - val_accuracy: 0.7729 - val_loss: 1.1949
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.8764 - loss: 0.6825 - val_accuracy: 0.9130 - val_loss: 0.5448
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.9430 - loss: 0.3136 - val_accuracy: 0.9469 - val_loss: 0.2736
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 0.9688 - loss: 0.1264 - val_accuracy: 0.9662 - val_loss: 0.1796
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 1.0000 - loss: 0.0590 - val_accuracy: 0.9855 - val_loss: 0.1298
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 1.0000 - loss: 0.0420 - val_accuracy: 0.9855 - val_loss: 0.1116
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 1.0000 - loss: 0.0246 - val_accuracy: 0.9855 - val_loss: 0.1024
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 1.0000 - loss: 0.0201 - val_accuracy: 0.9903 - val_loss: 0